In [ ]:
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams.update({
    "text.usetex": True,
    "font.family": "serif",
    "font.serif": ["Computer Modern Roman"],
})

In [ ]:
def draw_cell_lines(ax, n):
    """Draw cell boundary lines for an n x n grid"""
    for x in range(n + 1):
        ax.plot([x - 0.5, x - 0.5], [-0.5, n - 0.5], color='black', linewidth=1, zorder=1)
    for y in range(n + 1):
        ax.plot([-0.5, n - 0.5], [y - 0.5, y - 0.5], color='black', linewidth=1, zorder=1)

def plot_traversal(ax, coords, title, n):
    """Plot the traversal path given ordered (i, j) coordinates"""
    coords = np.array(coords)

    ax.set_xlim(-0.5, n-0.5)
    ax.set_ylim(-0.5, n-0.5)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_title(title, fontsize=24)
    ax.set_aspect('equal')

    # Turn off default grid and ticks
    ax.tick_params(left=False, bottom=False, labelleft=False, labelbottom=False)

    # Draw cell lines first
    draw_cell_lines(ax, n)

    # Then plot the red line above it
    ax.plot(coords[:,1], coords[:,0], '-', color='red', linewidth=4, zorder=2)

def z_order_coords(n):
    """Return a list of (i, j) coordinates in Z-order"""
    def interleave_bits(x, y):
        def split_by_1bit(v):
            v = (v | (v << 4)) & 0x0F0F
            v = (v | (v << 2)) & 0x3333
            v = (v | (v << 1)) & 0x5555
            return v
        return split_by_1bit(x) | (split_by_1bit(y) << 1)

    coords = []
    for i in range(n):
        for j in range(n):
            morton = interleave_bits(i, j)
            coords.append((i, j, morton))
    # Sort by Morton index
    coords_sorted = sorted(coords, key=lambda x: x[2])
    return [(i, j) for i, j, _ in coords_sorted]

n = 8

# Row-major coordinates
row_coords = [(i, j) for i in range(n) for j in range(n)]

# Column-major coordinates
col_coords = [(i, j) for j in range(n) for i in range(n)]

# Z-order coordinates
z_coords = z_order_coords(n)

# Plot all three
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

plot_traversal(axes[0], col_coords, 'Column Layout (F/LayoutRight)', n)
plot_traversal(axes[1], row_coords, 'Row Layout (C/LayoutLeft)', n)
plot_traversal(axes[2], z_coords, 'Morton Z-order Layout', n)

plt.tight_layout()

# Save as high-res PDF
fig.savefig('grid_traversals_2d.png', dpi=300, bbox_inches='tight')

plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Enable LaTeX rendering in matplotlib
plt.rcParams.update({
    "text.usetex": True,
    "font.family": "serif",
    "font.serif": ["Computer Modern Roman"],
})

# Parameters
r_c = 2.0  # cutoff radius
r = np.linspace(0.05, 5, 2000)  # higher resolution, avoid r=0

# Green's function
G = 1 / r

# Masks to exclude zero regions
mask_N = r < r_c
mask_F = r >= r_c

# Plot — shorter figure
plt.figure(figsize=(7, 3), dpi=300)

plt.plot(r[mask_N], G[mask_N], '-', color="#005f86", label=r'$G^n(\textbf{\emph{r}})$', linewidth=2, zorder=3)
plt.plot(r[mask_F], G[mask_F], '-', color="#f8971f", label=r'$G^f(\textbf{\emph{r}})$', linewidth=2, zorder=3)

# Vertical dashed line for r_c — higher zorder to be on top
plt.axvline(r_c, color='gray', linestyle='--', linewidth=1, zorder=5)
plt.text(r_c + 0.1, 1, r'$r_c$', fontsize=14, va='center', zorder=5)

# Labels and legend
plt.xlabel(r'$\|\textbf{\emph{r}}\|$', fontsize=14)
plt.ylabel(r'$G(\textbf{\emph{r}})$', fontsize=14)
plt.title(r'Splitting of $G(\textbf{\emph{r}}) = 1/\| \textbf{\emph{r}} \|$ in FMM', fontsize=16)
plt.legend(fontsize=12, loc='upper right')

# Limits
plt.xlim(0, 5)
plt.ylim(0, 6)

# Remove tick marks and labels
plt.xticks([])
plt.yticks([])

plt.tight_layout()
plt.savefig('fmm_green_split.png', dpi=300, bbox_inches='tight')
plt.show()